# Pytorch Implementation of a Keras Architecture 
by Amee Tan & Evan Chen 


### Task: 
Given pictures of handwritten names, predict the name that was written in the picture. 

### Goal:
Currently, there are no published Pytorch notebooks for this task. Our goal was to take an existing Keras architecture and implement it in Pytorch. The original Keras notebook can be found here: https://www.kaggle.com/samfc10/handwriting-recognition-using-crnn-in-keras


### Data: 
Training set: 331,059 images <br>
Validation set: 41,382 images <br>
Testing set: 41,382 images <br>
Source: https://www.kaggle.com/landlord/handwriting-recognition


In [1]:
!pip install torchinfo

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from torchinfo import summary

# Mixed Precision Training
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler


# Load Data

In [3]:
df_train = pd.read_csv('/kaggle/input/handwriting-recognition/written_name_train_v2.csv')
df_valid = pd.read_csv('/kaggle/input/handwriting-recognition/written_name_validation_v2.csv') # Locally
df_test = pd.read_csv('/kaggle/input/handwriting-recognition/written_name_test_v2.csv')

df_train.head()

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE


In [4]:
# Are there any null values? 

print("Number of nulls in train:", df_train['IDENTITY'].isnull().sum())
print("Number of nulls in valid:",df_valid['IDENTITY'].isnull().sum())

Number of nulls in train: 565
Number of nulls in valid: 78


In [5]:
# Drop the rows with null values for the label (IDENTITY column)

df_train.dropna(inplace=True)
df_valid.dropna(inplace=True)
df_test.dropna(inplace=True)

In [6]:
# How many unreadable images are there in each set? 
print(len(df_train.loc[df_train['IDENTITY']=='UNREADABLE']))
print(len(df_valid.loc[df_valid['IDENTITY']=='UNREADABLE']))
print(len(df_test.loc[df_test['IDENTITY']=='UNREADABLE']))

102
12
11


In [7]:
# Remove the unreadable images from the train and valid sets

df_train = df_train[df_train['IDENTITY'] != 'UNREADABLE']
df_valid = df_valid[df_valid['IDENTITY'] != 'UNREADABLE']
df_test = df_test[df_test['IDENTITY'] != 'UNREADABLE']

df_train.reset_index(inplace = True, drop=True) 
df_valid.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)



In [8]:
# There are some labels that are lowercase. Convert all labels to uppercase

df_train['IDENTITY'] = df_train['IDENTITY'].str.upper()
df_valid['IDENTITY'] = df_valid['IDENTITY'].str.upper()
df_test['IDENTITY'] = df_test['IDENTITY'].str.upper()

In [9]:
# How long is the longest name that we'll encounter? 

df_train['LABEL LENGTH'] = df_train['IDENTITY'].apply(lambda x: len(x))
df_valid['LABEL LENGTH'] = df_valid['IDENTITY'].apply(lambda x: len(x))
df_test['LABEL LENGTH'] = df_test['IDENTITY'].apply(lambda x: len(x))

print(df_train.describe()) # 34 for the training set 
print(df_valid.describe()) # 21 for the valid set
print(df_test.describe()) # 24 for the test set

        LABEL LENGTH
count  330294.000000
mean        6.546531
std         2.123296
min         1.000000
25%         5.000000
50%         6.000000
75%         7.000000
max        34.000000
       LABEL LENGTH
count  41280.000000
mean       6.556613
std        2.127069
min        1.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       21.000000
       LABEL LENGTH
count  41289.000000
mean       6.545860
std        2.137525
min        1.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       24.000000


# Prepare Images 

In [10]:
# Code borrowed from https://www.kaggle.com/samfc10/handwriting-recognition-using-crnn-in-keras

def preprocess(img):
    (h, w) = img.shape
    
    final_img = np.ones([64, 256])*255 # blank white image
    
    # Width and height are cropped if greater than 256x64; If smaler, image is padded with white pixesls
    if w > 256:
        img = img[:, :256]
        
    if h > 64:
        img = img[:64, :]
    
    
    final_img[:h, :w] = img
    # Rotate clockwise
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)

# Prepare Labels: Convert names into a sequence of integers

In [11]:
# Code adapted from same notebook as above 

alphabets = u"ABCDEFGHIJKLMNOPQRSTUVWXYZ-' "
max_str_len = 64 # max length of input labels
num_of_characters = len(alphabets) + 1 # +1 for ctc pseudo blank
num_of_timestamps = 64 # max length of predicted labels 

def label_to_num(label):
    label_num = []
    for ch in label:
        label_num.append(alphabets.find(ch))
        
    return np.array(label_num)

def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:  # CTC Blank
            break
        else:
            ret+=alphabets[ch]
    return ret

In [12]:
def encode_label(row):
    label = np.zeros(max_str_len)
    for i in range(max_str_len):
        label[0:len(row)] = label_to_num(row)

    return label
    

In [13]:
df_train['ENCODED LABEL'] = df_train['IDENTITY'].apply(lambda x: encode_label(x))
df_valid['ENCODED LABEL'] = df_valid['IDENTITY'].apply(lambda x: encode_label(x))

df_train

,FILENAME,IDENTITY,LABEL LENGTH,ENCODED LABEL
0,TRAIN_00001.jpg,BALTHAZAR,9,"[1.0, 0.0, 11.0, 19.0, 7.0, 0.0, 25.0, 0.0, 17..."
1,TRAIN_00002.jpg,SIMON,5,"[18.0, 8.0, 12.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0..."
2,TRAIN_00003.jpg,BENES,5,"[1.0, 4.0, 13.0, 4.0, 18.0, 0.0, 0.0, 0.0, 0.0..."
3,TRAIN_00004.jpg,LA LOVE,7,"[11.0, 0.0, 28.0, 11.0, 14.0, 21.0, 4.0, 0.0, ..."
4,TRAIN_00005.jpg,DAPHNE,6,"[3.0, 0.0, 15.0, 7.0, 13.0, 4.0, 0.0, 0.0, 0.0..."
...,...,...,...,...
330289,TRAIN_330957.jpg,LENNY,5,"[11.0, 4.0, 13.0, 13.0, 24.0, 0.0, 0.0, 0.0, 0..."
330290,TRAIN_330958.jpg,TIFFANY,7,"[19.0, 8.0, 5.0, 5.0, 0.0, 13.0, 24.0, 0.0, 0...."
330291,TRAIN_330959.jpg,COUTINHO DESA,13,"[2.0, 14.0, 20.0, 19.0, 8.0, 13.0, 7.0, 14.0, ..."
330292,TRAIN_330960.jpg,MOURAD,6,"[12.0, 14.0, 20.0, 17.0, 0.0, 3.0, 0.0, 0.0, 0..."


In [14]:
len(df_train)

330294

In [15]:
# Create a dataset 

class HandwritingDataset(Dataset):
    def __init__(self, df, folder_path):
        self.df = df
        self.folder_path = folder_path  # ex. '/content/train_v2/train/'
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # retrieve image
        path = self.folder_path+self.df.loc[idx,'FILENAME']
        
        # read the img
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        img = preprocess(img)
        
        # convert to [0,1] scale -> normalize
        img = torch.tensor(img / 255.).float()
        
        # Encode the label 
        label = torch.tensor(self.df.loc[idx,'ENCODED LABEL'])
        #label = torch.tensor(label_to_num(self.df.loc[idx,'IDENTITY'])) # Returns label as a sequence of numbers 
        label_length = self.df.loc[idx,'LABEL LENGTH']
        
        return img, label, label_length

## CNN Architecture --> RNN Arhitecture


In [16]:
class Mish(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return (x*torch.tanh(F.softplus(x)))

In [17]:
class CNN_RNN(nn.Module):
    """CNN and RNN model from class"""
    def __init__(self, mish=False):
        super().__init__()
        
        # same padding!
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        # pooling
        self.pool_1 = nn.MaxPool2d(kernel_size=2)
        self.pool_2 = nn.MaxPool2d(kernel_size=(1,2))
    
        # activation
        self.relu = nn.ReLU()
        if mish:
            self.relu = Mish()

        # batchnorm
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        # Linear Layer (Dense Layer)
        self.linear1 = nn.Linear(in_features=1024, out_features=64)
        self.linear2 = nn.Linear(in_features=1024, out_features=30)

        # RNN Layer --> Single LSTM with num_layers=2
        self.lstm1 = nn.LSTM(input_size=64, hidden_size=512, batch_first=True, bidirectional=True, num_layers=2)

        #self.unroll = nn.Flatten()
        
        #self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        # CNN
        # Start with image that is 256 wide x 64 tall and 1 channel
        # End with 64 wide x 8 tall and 128 channels
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool_1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool_1(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool_2(x)
        

        # CNN to RNN
        # Reshape to a sequence vector that is 64 wide and 1024 deep 
        batch_size = x.shape[0]

        x = torch.reshape(x,(batch_size,64,-1)) # or 1024 instead of -1?? 
        
        # Now we shrink the sequence vector to be 512 deep 
        x = self.linear1(x) 
        
        x = self.lstm1(x)[0] #[0] to get outputs, not hidden

        # OUTPUT
        x = self.linear2(x) # torch.Size([2, 2, 30])

        return x


In [18]:
model = CNN_RNN(mish=True)
summary(model, input_size = (2, 1, 256, 64), device='cpu')  # inputsize = (batch_size, channels, image length, image width)


Layer (type:depth-idx)                   Output Shape              Param #
CNN_RNN                                  --                        --
├─Conv2d: 1-1                            [2, 32, 256, 64]          320
├─BatchNorm2d: 1-2                       [2, 32, 256, 64]          64
├─Mish: 1-3                              [2, 32, 256, 64]          --
├─MaxPool2d: 1-4                         [2, 32, 128, 32]          --
├─Conv2d: 1-5                            [2, 64, 128, 32]          18,496
├─BatchNorm2d: 1-6                       [2, 64, 128, 32]          128
├─Mish: 1-7                              [2, 64, 128, 32]          --
├─MaxPool2d: 1-8                         [2, 64, 64, 16]           --
├─Conv2d: 1-9                            [2, 128, 64, 16]          73,856
├─BatchNorm2d: 1-10                      [2, 128, 64, 16]          256
├─Mish: 1-11                             [2, 128, 64, 16]          --
├─MaxPool2d: 1-12                        [2, 128, 64, 8]           --
├─Li

In [19]:
def one_pass(model, dataloader, optimizer, backwards=True, print_loss=True):
    
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)   
    lossFun = nn.CTCLoss()
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0
    total_correct_char = 0
    correct = 0
    correct_chars = 0
    total_chars = 0
    for img, labels, label_length in dataloader:
        
        # Send to GPU
        img = img.to(device)
        labels = labels.to(device)
        label_length = label_length.to(device)
        
        
        model.train()
        y_pred = model(img.unsqueeze(1))
        lsm = nn.LogSoftmax()
        y_pred = lsm(y_pred)

        yinput = y_pred.permute(1,0,2)  # input sequence length, batch_size, number of classes 

        N = labels.shape[0] # batch size 
        input_lengths = torch.ones(N,dtype=torch.long)*64

        loss = lossFun(yinput, labels, input_lengths, label_length)
        total_loss += loss.item()
        
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

  
        # ACCURACY 
        
        pred_nums = torch.argmax(y_pred, dim=2)
        
        # Character accuracy
        for pred, label, length in zip(pred_nums, labels, label_length):
            length = length.item()
            pred = torch.split(pred, length, dim=0)
            pred = pred[0]
            label = torch.split(label, length, dim=0)
            label = label[0]
            correct_chars += torch.sum(pred==label) 
            total_chars += length

                    
        # Check if words are same
        for i in range(N):
            pr = pred_nums[i]
            tr = labels[i]
            if torch.equal(pr, tr.long()):
                correct +=1
                
    avg_loss = total_loss / len(dataloader)
    avg_correct_chars = correct_chars/total_chars
    
    return avg_loss, avg_correct_chars, correct

In [20]:
def train_epochs_gpu(batch_size, lr, num_epochs):
    
    dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
    dl_valid = DataLoader(ds_valid, batch_size=batch_size, shuffle=False)
    
    # model with mish activation function
    model = CNN_RNN(mish=True)
    optimizer = optim.Adam(model.parameters(), lr=lr )

    model = model.to(device)

    train_losses = []
    valid_losses = []
    train_correct_chars = []
    valid_correct_chars = []
    train_correct_words = []
    valid_correct_words = []
    for epoch in range(num_epochs):
        print('Epoch: ', epoch)

        train_avg_loss, train_avg_correct_chars, train_num_correct_words = one_pass(model, dl_train, optimizer)
        train_losses.append(train_avg_loss)
        train_correct_chars.append(train_avg_correct_chars.item())
        train_correct_words.append(train_num_correct_words)
        print("Train:")
        print("CTC Loss:", round(train_avg_loss,4))
        print("Percent correct characters per word:", round(train_avg_correct_chars.item(),4))
        print("Number of correct words:", train_num_correct_words)


        valid_avg_loss, valid_avg_correct_chars, valid_num_correct_words = one_pass(model, dl_valid, optimizer, backwards=False)
        valid_losses.append(valid_avg_loss)
        valid_correct_chars.append(valid_avg_correct_chars.item())
        valid_correct_words.append(valid_num_correct_words)
        print("Valid")
        print("CTC Loss", round(valid_avg_loss,4))
        print("Percent correct characters per word", round(valid_avg_correct_chars.item(),4))
        print("Number of correct words", valid_num_correct_words)
        print("")
        
    return train_losses, valid_losses, valid_correct_chars, valid_correct_words

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device(0)

In [22]:
# Create mini datasets for testing

df_mini_train = df_train.iloc[:1000]
ds_train = HandwritingDataset(df_mini_train, '/kaggle/input/handwriting-recognition/train_v2/train/')

df_mini_valid = df_valid.iloc[:100]
ds_valid = HandwritingDataset(df_mini_valid, '/kaggle/input/handwriting-recognition/validation_v2/validation/')


In [23]:
# Create dataloaders 

dl_train = DataLoader(ds_train, batch_size = 32, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size = 32, shuffle=False)

In [24]:
# Test functions on mini dataset 

train_losses, valid_losses, valid_correct_chars, valid_correct_words = train_epochs_gpu(batch_size=32, lr=0.001, num_epochs=5)

Epoch:  0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train:
CTC Loss: 26.2691
Percent correct characters per word: 0.0809
Number of correct words: 0
Valid
CTC Loss 20.5455
Percent correct characters per word 0.0949
Number of correct words 0

Epoch:  1
Train:
CTC Loss: 25.0529
Percent correct characters per word: 0.0683
Number of correct words: 0
Valid
CTC Loss 19.937
Percent correct characters per word 0.0735
Number of correct words 0

Epoch:  2
Train:
CTC Loss: 23.9254
Percent correct characters per word: 0.0772
Number of correct words: 0
Valid
CTC Loss 20.1873
Percent correct characters per word 0.0582
Number of correct words 0

Epoch:  3
Train:
CTC Loss: 24.5009
Percent correct characters per word: 0.0659
Number of correct words: 0
Valid
CTC Loss 19.8191
Percent correct characters per word 0.0628
Number of correct words 0

Epoch:  4
Train:
CTC Loss: 23.7907
Percent correct characters per word: 0.0623
Number of correct words: 0
Valid
CTC Loss 20.1862
Percent correct characters per word 0.0597
Number of correct words 0



In [25]:
print(train_losses)
print(valid_losses)
print(valid_correct_chars)
print(valid_correct_words) 

[26.26910811662674, 25.052927374839783, 23.925353676080704, 24.500934571027756, 23.790739744901657]
[20.545480370521545, 19.937028467655182, 20.187299728393555, 19.81905573606491, 20.186236143112183]
[0.09494640678167343, 0.07350689172744751, 0.05819295719265938, 0.06278713792562485, 0.0597243495285511]
[0, 0, 0, 0, 0]


In [26]:
# Train for real on 1 epochs --> started at 8:47am on Tuesday

ds_train = HandwritingDataset(df_train, '/kaggle/input/handwriting-recognition/train_v2/train/')
ds_valid = HandwritingDataset(df_valid, '/kaggle/input/handwriting-recognition/validation_v2/validation/')

dl_train = DataLoader(ds_train, batch_size = 32, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size = 32, shuffle=False)

train_losses, valid_losses, valid_correct_chars, valid_correct_words = train_epochs_gpu(batch_size=32, lr=0.001, num_epochs=20)

Epoch:  0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train:
CTC Loss: 24.0105
Percent correct characters per word: 0.0697
Number of correct words: 29


KeyboardInterrupt: 

In [ ]:
# Train for real on 20 epochs --> started at 8:47am on Tuesday

ds_train = HandwritingDataset(df_train, '/kaggle/input/handwriting-recognition/train_v2/train/')
ds_valid = HandwritingDataset(df_valid, '/kaggle/input/handwriting-recognition/validation_v2/validation/')

dl_train = DataLoader(ds_train, batch_size = 32, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size = 32, shuffle=False)

train_losses, valid_losses, valid_correct_chars, valid_correct_words = train_epochs_gpu(batch_size=32, lr=0.001, num_epochs=20)

Epoch:  0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train:
CTC Loss: 24.1788
Percent correct characters per word: 0.0704
Number of correct words: 14
Valid
CTC Loss 22.5502
Percent correct characters per word 0.0836
Number of correct words 1

Epoch:  1
Train:
CTC Loss: 21.6404
Percent correct characters per word: 0.1021
Number of correct words: 7
Valid
CTC Loss 20.377
Percent correct characters per word 0.1222
Number of correct words 0

Epoch:  2
Train:
CTC Loss: 19.6929
Percent correct characters per word: 0.1265
Number of correct words: 8
Valid
CTC Loss 18.1404
Percent correct characters per word 0.1396
Number of correct words 0

Epoch:  3
Train:
CTC Loss: 17.1576
Percent correct characters per word: 0.1613
Number of correct words: 3
Valid
CTC Loss 16.0976
Percent correct characters per word 0.1783
Number of correct words 1

Epoch:  4
Train:
CTC Loss: 15.2663
Percent correct characters per word: 0.1828
Number of correct words: 7
Valid
CTC Loss 14.5922
Percent correct characters per word 0.1861
Number of correct words 2

Epoch:  5
Train